# Práctica Dos

Grupo 14:
* Joaquín Ibáñez Penalva
* Aurora Zuoris

Para la realización de esta práctica  se usará la librería de numpy y sklearn.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [ ]:
df = pd.read_csv('tiempos.csv', sep=';')
df.head()

In [ ]:
def hhmmss_to_mins(x):
	h, m, s = map(int, x.split(':'))
	return h * 60 + m + s / 60

df = df.applymap(hhmmss_to_mins)
df.head()

In [ ]:
data = df.to_numpy()
data.shape

## Ejercicio 1

In [ ]:
train, test = train_test_split(data, test_size=0.3) # 70% train, 30% test
x = train[:, :-1] # todas las filas, todas las columnas menos la última
y = train[:, -1] # todas las filas, la última columna
reg = LinearRegression().fit(x, y) 
reg.score(x, y) 
pesos = reg.coef_ # coeficientes
intercepcion = reg.intercept_ # intercepción
prediccion = reg.predict(test[:, :-1]) # predicción
print(pesos)
print(reg.intercept_)

reg2 = LinearRegression(fit_intercept=False).fit(x, y) # sin intercepción
reg2.score(x, y)
pesos2 = reg2.coef_ 
print(pesos2)

In [ ]:
ax = plt.subplot(111)
ax.bar( np.arange(len(pesos) + 1), np.append(pesos, intercepcion), width=0.3) # pesos + intercepción
ax.bar( np.arange(len(pesos2))-0.3, pesos2, width=0.3) # pesos sin intercepción
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_xticks(np.arange(len(pesos) + 1), ['P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'w0']) 
ax.legend(['w0 != 0', 'w0 = 0'])
ax.set_title('Gráfico de Barras')
ax.grid()
plt.show()

Gráfico cuando la intercepción (w0) se estima o cuando es 0. Se puede apreciar como en P6 suele ser practicamente igual, mientras que en P1 o P2 suele variar más.

In [ ]:
mean_squared_error(prediccion, test[:, -1], squared=False) # error cuadrático medio

## Ejercicio 2